In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import norm
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, Attention
from tensorflow.keras.models import Model
from ta.volume import MFIIndicator
from ta.utils import dropna
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [12]:
# Kaggle function

def make_model():
    inp = Input(shape=(128, 10))
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = Attention(128)(x)
    # A intermediate full connected (Dense) can help to deal with nonlinears outputs
    x = Dense(64, activation="relu")(x)
    x = Dense(9, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [29]:
# My function
def make_model():
    # 1. Input layer for stock data (sequence_length, features)
    inp = Input(shape=(128, 9))  # Example: 128 timesteps, 10 features (adjust as needed)

    # 2. Bidirectional LSTM layers for sequence learning
    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    # 3. Dropout to prevent overfitting
    x = Dropout(0.2)(x)

    # 4. 1D Convolutional layer for feature extraction
    x = Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)

    # 5. MaxPooling to downsample
    x = MaxPooling1D(pool_size=2)(x)

    # 6. Flatten layer to transition to fully connected layers
    x = Flatten()(x)

    # 7. Fully connected layers for high-level learning
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)

    # 8. Output layer - Single neuron with tanh activation to produce values between -1 and 1
    output = Dense(1, activation="tanh")(x)

    # 9. Compile model with Adam optimizer
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    return model

# Create the model
model = make_model()
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 128, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 128, 256)       │       141,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 128, 128)       │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 128, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 863,041 (3.29 MB)

 Trainable params: 863,041 (3.29 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    rsi = rsi.fillna(50)  # Default to neutral if not enough data
    return rsi

In [31]:
def get_stock_data(ticker, start='2024-01-01', end='2025-03-24', buffer_days=200):
    # Extend the start date backward by the buffer_days
    extended_start = pd.to_datetime(start) - pd.Timedelta(days=buffer_days)
    stock = yf.download(ticker, start=extended_start.strftime('%Y-%m-%d'), end=end)
    return stock

In [32]:
def get_dividends(ticker):
    session = requests.session()
    stock  = yf.Ticker(ticker, session = session)
    dividends = stock.dividends
    print(dividends)

In [33]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

In [ ]:


def data_preparation(data, start_date='2024-01-01', buffer_days_ma50=50, buffer_days_ma200=200):
    data_ma50 = data[data.index >= (pd.to_datetime(start_date) - pd.Timedelta(days=buffer_days_ma50))]
    data_ma200 = data[data.index >= (pd.to_datetime(start_date) - pd.Timedelta(days=buffer_days_ma200))]
    print(data_ma50.shape, data_ma200.shape)
    print(f"Initial data shape before preparation: {data.shape}")

    data = dropna(data)  # Drop NA values from the dataframe
    print(data.shape)
    data[['Log Price']] = np.log(data['Close'])  # Log transformation for stationarity
    data[['Log Volume']] = np.log(data['Volume'].replace(0,np.nan))  # Log transformation for volume, add 1 to avoid log(0)
    data.loc[:, 'Log Volume'] = data['Log Volume'].fillna(data['Log Volume'].rolling(window=5, min_periods=1).mean())
    data[['Log Price Diff']] = data[['Log Price']].diff()

    data['Percent Change'] = data['Close'].pct_change()  # Calculate percentage change for the close price

    data['RSI'] = calculate_rsi(data)  # Calculate RSI
    
    mfi = MFIIndicator(
        high=data['High'].squeeze(),
        low=data['Low'].squeeze(),
        close=data['Close'].squeeze(),
        volume=data['Volume'].squeeze(),
        window=14,
        fillna=True)
    data['MFI'] = mfi.money_flow_index()
    
    data['Log Volume Diff'] = data['Log Volume'].diff()  # Log volume difference for stationarity
    
    # Beta calculation (S&P500)
    market = yf.download('SPY', start=data.index[0], end=data.index[-1])['Close']
    data['Market Return'] = market.pct_change()
    data['Stock Return'] = data['Close'].pct_change()
    data['Beta'] = data['Stock Return'].rolling(30).cov(data['Market Return']) / data['Market Return'].rolling(30).var()
    
    data['PE Ratio'] = data['Close'] / data['Earnings'] if 'Earnings' in data.columns else np.nan
     # Example PE ratio calculation, ensure 'Earnings' column exists in your data

    data_ma50.loc[:, 'MA50'] = data_ma50['Close'].rolling(50).mean()
    data_ma50.loc[:, 'Log Diff MA50'] = np.log(data_ma50['Close'].values.flatten()) - np.log(data_ma50['MA50'].values.flatten())
    data_ma200.loc[:,'MA200'] = data_ma200['Close'].rolling(200).mean()
    data_ma200.loc[:,'Log Diff MA200'] = np.log(data_ma200['Close'].values.flatten()) - np.log(data_ma200['MA200'].values.flatten())

    data_ma50 = data_ma50.dropna()  # Drop rows where MA50 or Log Diff MA50 is NaN
    data_ma200 = data_ma200.dropna()  # Drop rows where MA200 or Log Diff MA200 is NaN

    print(f"Final MA50 data shape after preparation: {data_ma50.shape}")
    print(f"Final MA200 data shape after preparation: {data_ma200.shape}")



    decomposition = seasonal_decompose(data['Close'], model='additive', period=math.floor(data['Close'].size/2))  # Assuming yearly seasonality
    data['Seasonality'] = decomposition.seasonal

    data = data[data.index >= start_date]
    print(f"Final data shape after preparation: {data.shape}")
    data = data.merge(data_ma50[['MA50', 'Log Diff MA50']], how='left', left_index=True, right_index=True)
    data = data.merge(data_ma200[['MA200', 'Log Diff MA200']], how='left', left_index=True, right_index=True)
    print(f"Final data shape after preparation: {data.shape}")

    feature_columns = ['Log Price Diff',  'RSI', 'MFI', 'Log Volume Diff',
                       'Beta', 'PE Ratio', 'Log Diff MA50', 'Log Diff MA200', 'Seasonality']
    
    return data[feature_columns],data['Percent Change']

    # Normalize the data
    #data = (data - data.mean()) / data.std()
    

In [48]:
apple = get_stock_data('AAPL')
prepped_data,target = data_preparation(apple)
prepped_data.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

(339, 5) (443, 5)
Initial data shape before preparation: (443, 5)
(443, 5)



/var/folders/39/953q32fx2pgd35pdr0s2p0r00000gn/T/ipykernel_5506/4267602873.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ma50.loc[:, 'MA50'] = data_ma50['Close'].rolling(50).mean()
/var/folders/39/953q32fx2pgd35pdr0s2p0r00000gn/T/ipykernel_5506/4267602873.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ma50.loc[:, 'Log Diff MA50'] = np.log(data_ma50['Close'].values.flatten()) - np.log(data_ma50['MA50'].values.flatten())


KeyError: ['MA200', 'Log Diff MA200']

In [21]:
def create_sequences(data, target, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Sequence of features
        y.append(target[i+seq_length])  # Corresponding target value
    return np.array(X), np.array(y)



In [22]:
seq_length = 128 # Number of time steps to look back
X, y = create_sequences(prepped_data.values, target.values, seq_length)

In [23]:


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # 20% val, 10% test


In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

c:\Users\evany\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\evany\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\evany\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


In [25]:
BATCH_SIZE = 32  # Adjust based on dataset size

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).shuffle(1000)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)


In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  
              loss='mse',   # Mean Squared Error (common for regression)
              metrics=['mae', 'mape'])  # Mean Absolute Error & Mean Absolute Percentage Error


In [27]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=32, 
                    verbose=1)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 398ms/step - loss: 2.0820e-04 - mae: 0.0102 - mape: 99.0579 - val_loss: 1.9694e-04 - val_mae: 0.0110 - val_mape: 12980.7158
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - loss: 2.3001e-04 - mae: 0.0111 - mape: 109.0237 - val_loss: 2.0646e-04 - val_mae: 0.0115 - val_mape: 45177.8242
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - loss: 1.9489e-04 - mae: 0.0103 - mape: 134.2506 - val_loss: 2.0420e-04 - val_mae: 0.0114 - val_mape: 38277.8477
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - loss: 2.2916e-04 - mae: 0.0112 - mape: 118.8020 - val_loss: 2.0615e-04 - val_mae: 0.0115 - val_mape: 44239.5859
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - loss: 2.2259e-04 - mae: 0.0110 - mape: 127.5286 - val_loss: 2.0418e-04 - val_mae: 0.0114 - val_mape: 38208.4609
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - loss: 2.2838e-04 - mae: 0.0110 - mape: 129.6214 - val_loss: 2.0176e-04 - val_mae: 0.0113 - val_mape: 30409.4238
Epoch 7/50
4/4 ━━